In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Use 'google/mt5-small' for non-pro cloab users
model_repo = 'google/mt5-base'
model_path = 'mt5_translation.pt'
max_seq_len = 20

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
from datasets import load_dataset, DatasetDict

# Load the 'alt' dataset
dataset = load_dataset('alt')


ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
def process(x):
    return {'bg': x['bg'], 'en': x['en'], 'hi': x['hi']}


In [ ]:
train_df = pd.DataFrame(dataset["train"])
train_df["translation"] = train_df["translation"].apply(lambda col: process(col))

test_df = pd.DataFrame(dataset["test"])
test_df["translation"] = test_df["translation"].apply(lambda col: process(col))

val_df = pd.DataFrame(dataset["validation"])
val_df["translation"] = val_df["translation"].apply(lambda col: process(col))


dataset["train"] = dataset["train"].from_pandas(train_df)
dataset["test"] = dataset["test"].from_pandas(test_df)
dataset["validation"] = dataset["validation"].from_pandas(val_df)
dataset

DatasetDict({
    train: Dataset({
        features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
        num_rows: 18088
    })
    validation: Dataset({
        features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
        num_rows: 1019
    })
})

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'hi': '<hi>',
    'bg': '<bg>'
}

In [5]:

special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

NameError: name 'tokenizer' is not defined